In [17]:
!pip install geocoder
!pip install folium

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import folium

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,

## Part 1: Scrape Toronto neighborhood data from Wikipedia page

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')


table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
clean_nbh = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        clean_nbh.append([postalcode, borough, neighborhood])


In [28]:
cols = ['Postal Code', 'Borough', 'Neighborhood']
NBH_DF = pd.DataFrame(clean_nbh, columns=cols)

In [29]:
NBH_DF.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [7]:
print(NBH_DF.shape)

(212, 3)


## Part 2:  Obtain Latitudes and Longitudes via geocoder

In [27]:
to_drop_unknown = []
geolocator = Nominatim(user_agent="coursera")
for index, row in NBH_DF.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geographical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        NBH_DF.loc[index, 'Latitude'] = latitude
        NBH_DF.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

NBH_DF.head()


The geographical coordinate of Parkwoods, Toronto are 43.7612239, -79.3239857.
The geographical coordinate of Victoria Village, Toronto are 43.732658, -79.3111892.


GeocoderUnavailable: Service not available

## Since I don't really understand the Geocoder or the Nominatim code, I am using the CSV file to create the latitude longitude dataframe.

In [24]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2019-02-27 19:21:33--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-02-27 19:21:33--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-27 19:21:36--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-27 19:21:36--  https://ibm.box.com/public/static/9afzr83pps4pwf2sm

In [31]:
LL_df = pd.read_csv('GeoCord.csv')
LL_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
NBH_DF['Latitude'] = np.nan
NBH_DF['Longitude'] = np.nan

for idx in NBH_DF.index:
    coord_idx = LL_df['Postal Code'] == NBH_DF.loc[idx, 'Postal Code']
    NBH_DF.at[idx, 'Latitude'] = LL_df.loc[coord_idx, 'Latitude'].values
    NBH_DF.at[idx, 'Longitude'] = LL_df.loc[coord_idx, 'Longitude'].values
    
NBH_DF.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353
